# [Residual Networks](https://arxiv.org/pdf/1512.03385)

- 2015 ILSVRC 1st place
- ResNet-50
- Residual Module
- BottleNeck Module

![alt text](./networks/resnet.jpg)
![alt text](./networks/resnet_architecture.png)

## 1. Settings
### 1) Import required libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

### 2) Hyperparameter

In [2]:
batch_size= 1
learning_rate = 0.0002
epoch = 100

## 2. Data Loader

In [3]:
img_dir = "./images"
img_data = dset.ImageFolder(img_dir, transforms.Compose([
            transforms.Scale(256),
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            ]))

img_batch = data.DataLoader(img_data, batch_size=batch_size,
                            shuffle=True, num_workers=2)

## 3. Model 
### 1) Basic Block

In [4]:
def conv_block_1(in_dim,out_dim,act_fn):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=1, stride=1),
        act_fn,
    )
    return model

def conv_block_1_stride_2(in_dim,out_dim,act_fn):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=1, stride=2),
        act_fn,
    )
    return model

def conv_block_1_n(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=1, stride=1),
    )
    return model

def conv_block_1_stride_2_n(in_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=1, stride=2),
    )
    return model

def conv_block_3(in_dim,out_dim,act_fn):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim, kernel_size=3, stride=1, padding=1),
        act_fn,
    )
    return model

### 2) Bottle Neck Module

In [5]:
class BottleNeck(nn.Module):
    
    def __init__(self,in_dim,mid_dim,out_dim,act_fn):
        super(BottleNeck,self).__init__()
        self.layer = nn.Sequential(
            conv_block_1(in_dim,mid_dim,act_fn),
            conv_block_3(mid_dim,mid_dim,act_fn),
            conv_block_1_n(mid_dim,out_dim),
        )
        self.downsample = nn.Conv2d(in_dim,out_dim,1,1)
        
    def forward(self,x):
        downsample = self.downsample(x)
        out = self.layer(x)
        out = out + downsample
        
        return out

    
class BottleNeck_no_down(nn.Module):
    
    def __init__(self,in_dim,mid_dim,out_dim,act_fn):
        super(BottleNeck_no_down,self).__init__()
        self.layer = nn.Sequential(
            conv_block_1(in_dim,mid_dim,act_fn),
            conv_block_3(mid_dim,mid_dim,act_fn),
            conv_block_1_n(mid_dim,out_dim),
        )
        
    def forward(self,x):
        out = self.layer(x)
        out = out + x
        
        return out

    
class BottleNeck_stride(nn.Module):
    
    def __init__(self,in_dim,mid_dim,out_dim,act_fn):
        super(BottleNeck_stride,self).__init__()
        self.layer = nn.Sequential(
            conv_block_1_stride_2(in_dim,mid_dim,act_fn),
            conv_block_3(mid_dim,mid_dim,act_fn),
            conv_block_1_n(mid_dim,out_dim),
        )
        self.downsample = nn.Conv2d(in_dim,out_dim,1,2)
        
    def forward(self,x):
        downsample = self.downsample(x) 
        out = self.layer(x)
        out = out + downsample
        
        return out

### 2) ResNet Model

In [6]:
class ResNet(nn.Module):

    def __init__(self, base_dim, num_classes=2):
        super(ResNet, self).__init__()
        self.act_fn = nn.ReLU()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,base_dim,7,2,3),
            nn.ReLU(),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_2 = nn.Sequential(
            BottleNeck(base_dim,base_dim,base_dim*4,self.act_fn),
            BottleNeck_no_down(base_dim*4,base_dim,base_dim*4,self.act_fn),
            BottleNeck_stride(base_dim*4,base_dim,base_dim*4,self.act_fn),
        )   
        self.layer_3 = nn.Sequential(
            BottleNeck(base_dim*4,base_dim*2,base_dim*8,self.act_fn),
            BottleNeck_no_down(base_dim*8,base_dim*2,base_dim*8,self.act_fn),
            BottleNeck_no_down(base_dim*8,base_dim*2,base_dim*8,self.act_fn),
            BottleNeck_stride(base_dim*8,base_dim*2,base_dim*8,self.act_fn),
        )
        self.layer_4 = nn.Sequential(
            BottleNeck(base_dim*8,base_dim*4,base_dim*16,self.act_fn),
            BottleNeck_no_down(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
            BottleNeck_no_down(base_dim*16,base_dim*4,base_dim*16,self.act_fn),            
            BottleNeck_no_down(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
            BottleNeck_no_down(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
            BottleNeck_stride(base_dim*16,base_dim*4,base_dim*16,self.act_fn),
        )
        self.layer_5 = nn.Sequential(
            BottleNeck(base_dim*16,base_dim*8,base_dim*32,nn.ReLU()),
            BottleNeck_no_down(base_dim*32,base_dim*8,base_dim*32,self.act_fn),
            BottleNeck(base_dim*32,base_dim*8,base_dim*32,self.act_fn),
        )
        self.avgpool = nn.AvgPool2d(7,1) 
        self.fc_layer = nn.Linear(base_dim*32,num_classes)
        
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        out = self.layer_5(out)
        out = self.avgpool(out)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        
        return out
    
model = ResNet(base_dim=64).cuda()

for i in model.children():
    print(i)

ReLU ()
Sequential (
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): ReLU ()
  (2): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
)
Sequential (
  (0): BottleNeck (
    (layer): Sequential (
      (0): Sequential (
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU ()
      )
      (1): Sequential (
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU ()
      )
      (2): Sequential (
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (downsample): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (1): BottleNeck_no_down (
    (layer): Sequential (
      (0): Sequential (
        (0): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU ()
      )
      (1): Sequential (
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU ()
      )
      (2): Sequential (
  

## 4. Optimizer & Loss

In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

## 5. Train

In [8]:
for i in range(epoch):
    for img,label in img_batch:
        img = Variable(img).cuda()
        label = Variable(label).cuda()

        optimizer.zero_grad()
        output = model(img)
        
        loss = loss_func(output,label)
        loss.backward()
        optimizer.step()

    if i % 10 ==0:
        print(loss)
        #print(output.size())

Variable containing:
 0.5939
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.8869
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 1.5539
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 1.2280
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.3712
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.2974
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 1.4653
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 1.0721
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 1.0059
[torch.cuda.FloatTensor of size 1 (GPU 0)]



Process Process-191:
Process Process-192:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 35, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_

KeyboardInterrupt: 